In [56]:
import numpy as np
import plotly.express as px
from tools.utils import query_database
import pandas as pd
import psycopg2
import cv2
import matplotlib.pyplot as plt
import plotly.graph_objects as go

**Graphs**

- best models
- best accuracy
- best preprocesing
- loss
- accuracy


# Top Performing


In [69]:
query = """
with noShot as (
select
    m.name, m.preprocessing, d.shot, d.shuffle, d.optimiser, e.epoch, r.value
from
    model m
    join cnn_data d on m.id = d.model_id
    join epoch e on m.id = e.model_id
    join cnn_result r on e.id = r.epoch_id
where 
    m.type = 'cnn' and d.shot is null and r.type = 'Accuracy' and d.optimiser = 'adam'
order by
    r.value desc
limit 1
),
fiveShot as (
select
    m.name, m.preprocessing, d.shot, d.shuffle, d.optimiser, e.epoch, r.value
from
    model m
    join cnn_data d on m.id = d.model_id
    join epoch e on m.id = e.model_id
    join cnn_result r on e.id = r.epoch_id
where 
    m.type = 'cnn' and d.shot = 5 and r.type = 'Accuracy' and d.optimiser = 'adam'
order by
    r.value desc
limit 1
),
tenShot as (
select
    m.name, m.preprocessing, d.shot, d.shuffle, d.optimiser, e.epoch, r.value
from
    model m
    join cnn_data d on m.id = d.model_id
    join epoch e on m.id = e.model_id
    join cnn_result r on e.id = r.epoch_id
where 
    m.type = 'cnn' and d.shot = 10 and r.type = 'Accuracy' and d.optimiser = 'adam'
order by
    r.value desc
limit 1
),
fifthteenShot as (
select
    m.name, m.preprocessing, d.shot, d.shuffle, d.optimiser, e.epoch, r.value
from
    model m
    join cnn_data d on m.id = d.model_id
    join epoch e on m.id = e.model_id
    join cnn_result r on e.id = r.epoch_id
where 
    m.type = 'cnn' and d.shot = 15 and r.type = 'Accuracy' and d.optimiser = 'adam'
order by
    r.value desc
limit 1
),
combined as (
select * from noShot
union
select * from fiveShot
union
select * from tenShot
union
select * from fifthteenShot
)
select
    concat_ws(', ', name, preprocessing, optimiser, shuffle, epoch) as x_value, value as y_value
from combined
"""

columns, rows = query_database(query)

df = pd.DataFrame(rows, columns=columns)
df["y_value"] = df["y_value"].astype(float)
df["x_value"] = df["x_value"].astype(str)
df

,x_value,y_value
0,"resnet18_transfer, cqt, adam, t, 36",0.589558
1,"resnet18, combined2, adam, 29",0.983839
2,"resnet18_transfer, cqt, adam, t, 40",0.670566
3,"resnet18, cqt, adam, f, 31",0.613323


In [70]:
x = df["x_value"]
y = df["y_value"]

fig = go.Figure(
    data=[
        go.Bar(
            x=x,
            y=y,
            text=y,
            textposition="auto",
        )
    ]
)
fig.update_layout(
    title="Top Performing Models of Each Shot",
    xaxis_title="Shot",
    yaxis_title="Accuracy",
)
fig.show()

# Heatmap


In [116]:
query = """
with cnn_top as (
select
    max(cr.value) as max, cd.shot
from
    model m
    join epoch e on m.id = e.model_id
    join cnn_result cr on e.id = cr.epoch_id
    join cnn_data cd on m.id = cd.model_id
where
    cr.type = 'Accuracy' and cd.shot is not null
group by
    m.id, cd.shot
),
maml_top as (
select
    max(mr.accuracy) as max
from
    model m
    join epoch e on m.id = e.model_id
    join maml_step ms on e.id = ms.epoch_id
    join maml_update_acc mr on ms.id = mr.maml_step_id
group by
    m.id
)
select
    *
from
    cnn_top
"""


columns, rows = query_database(query)

df = pd.DataFrame(rows, columns=columns)

df["index"] = df.index
df["max"] = df["max"].astype(float)
df["shot"] = df["shot"].astype(int)

df

,max,shot,index
0,0.285127,15,0
1,0.558838,5,1
2,0.491628,5,2
3,0.321777,15,3
4,0.454787,5,4
...,...,...,...
379,0.418574,15,379
380,0.311975,15,380
381,0.263768,5,381
382,0.243363,5,382


In [118]:
values = df["max"].tolist()

shot = [5, 10, 15]
shot_counts = {s: {} for s in shot}

# Initialize a dictionary to hold counts for each interval
interval_counts = {}

# Define the interval width
interval_width = 0.05

# Iterate over the values and count them in intervals
for value in values:
    interval = int(value / interval_width) * interval_width  # Determine the interval
    if interval not in interval_counts:
        interval_counts[interval] = 1
    else:
        interval_counts[interval] += 1

# Print the counts for each interval
for interval, count in sorted(interval_counts.items()):
    print(f"{interval:.2f}-{interval + interval_width:.2f}: {count}")

interval_counts

0.20-0.25: 59
0.25-0.30: 60
0.30-0.35: 39
0.35-0.40: 41
0.40-0.45: 37
0.45-0.50: 41
0.50-0.55: 44
0.55-0.60: 45
0.60-0.65: 17
0.65-0.70: 1


{0.25: 60,
 0.55: 45,
 0.45: 41,
 0.30000000000000004: 39,
 0.2: 59,
 0.6000000000000001: 17,
 0.5: 44,
 0.35000000000000003: 41,
 0.4: 37,
 0.65: 1}

In [107]:
x_axis = [i / len(df) - 1 for i in range(len(df))]

fig = go.Figure(
    data=go.Heatmap(
        x=x_axis,
        y=df["index"],
        z=df["max"],
    )
)


fig.show()

In [103]:
import plotly.graph_objects as go
import numpy as np

# Generate some random data (replace this with your actual data)
num_models = 450
accuracy_values = np.random.rand(num_models)

# Reshape accuracy values into a matrix
matrix_size = int(np.ceil(np.sqrt(num_models)))
accuracy_matrix = np.reshape(accuracy_values, (matrix_size, matrix_size))

# Create heatmap
fig = go.Figure(
    data=go.Heatmap(
        z=accuracy_matrix,
        colorscale="Viridis",  # You can choose other color scales as well
        zmin=0,  # Minimum value of the color scale
        zmax=1,  # Maximum value of the color scale
        colorbar=dict(title="Accuracy"),
    )
)

# Add title and labels
fig.update_layout(
    title="Accuracy of 450 Models", xaxis=dict(title="Model"), yaxis=dict(title="Model")
)

# Show the plot
fig.show()

ValueError: cannot reshape array of size 450 into shape (22,22)

# Mel
